In [1]:
import os
from os.path import join, pardir, curdir
import glob
import numpy as np
import pandas as pd
import scipy as sp
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

# Set current directory to "src"
os.chdir(join(os.getcwd(), os.pardir, "src"))
print(f"Current working directory: {os.getcwd()}")

from feature_extractor import *
from process import *

Current working directory: /home/fa926284/Documents/Parkinson_FOG_Detection/src


In [2]:
DATA_PATH = join(pardir, 'data', 'train')
DATASETS = ['tdcsfog', 'defog']


In [3]:
X_df = []
y = []

for dataset in DATASETS:
    print(f"\n\nProcessing {dataset} dataset ... ", end='\n\n')
    file_dirs = glob.glob(join(DATA_PATH, dataset, "*.csv"))
    for file_dir in tqdm(file_dirs):
        df = pd.read_csv(file_dir)
        
        if dataset == 'defog':
            df = df[(df['Valid'] == True) & (df['Task'] == True)]
            df = df.drop(['Valid', 'Task'], axis=1).reset_index()

        df['label'] = (df['StartHesitation'] * 1) | (df['Turn'] * 2) | (df['Walking'] * 3)
        
        acc_ap_win, label = get_windowed_data(df['AccAP'].to_numpy(),
                                              df['label'].to_numpy(),
                                              win_len=100, slide_frac=0.5)
        acc_v_win, _ = get_windowed_data(df['AccV'].to_numpy(),
                                         df['label'].to_numpy(),
                                         win_len=100, slide_frac=0.5)
        acc_ml_win, _ = get_windowed_data(df['AccML'].to_numpy(),
                                          df['label'].to_numpy(),
                                          win_len=100, slide_frac=0.5)
        
        acc_ap_stat = get_stat_features(acc_ap_win, prefix='AccAP')
        acc_v_stat = get_stat_features(acc_v_win, prefix='AccV')
        acc_ml_stat = get_stat_features(acc_ml_win, prefix='AccML')
        acc_ap_freq = get_freq_features(acc_ap_win, axis=1, fs=128, nperseg=20, prefix='AccAP')
        acc_v_freq = get_freq_features(acc_v_win, axis=1, fs=128, nperseg=20, prefix='AccV')
        acc_ml_freq = get_freq_features(acc_ml_win, axis=1, fs=128, nperseg=20, prefix='AccML')

        features = pd.concat([acc_ap_stat,
                              acc_v_stat,
                              acc_ml_stat,
                              acc_ap_freq,
                              acc_v_freq,
                              acc_ml_freq], axis=1)

        X_df.append(features)
        y.append(label)

X_df = pd.concat(X_df)
X = X_df.to_numpy()
y = np.concatenate(y)



Processing tdcsfog dataset ... 



100%|██████████| 833/833 [00:41<00:00, 20.18it/s]




Processing defog dataset ... 



  3%|▎         | 3/91 [00:00<00:31,  2.82it/s]/home/fa926284/Documents/Parkinson_FOG_Detection/src/feature_extractor.py:56: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skw = sp.stats.skew(x, axis=axis)
/home/fa926284/Documents/Parkinson_FOG_Detection/src/feature_extractor.py:57: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kut = sp.stats.kurtosis(x, axis=axis)
  4%|▍         | 4/91 [00:01<00:30,  2.84it/s]/home/fa926284/Documents/Parkinson_FOG_Detection/src/feature_extractor.py:56: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skw = sp.stats.skew(x, axis=axis)
/home/fa926284/Documents/Parkinson_FOG_Detection/